In [18]:
import os, re, json
from LiteLLM.common import CONFIG

from phoenix.evals import (
    llm_classify,
    LiteLLMModel,
    llm_generate,
    RelevanceEvaluator,
    run_evals
)

import litellm
litellm._turn_on_debug()

os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://localhost:6006"
os.environ["HUGGINGFACE_API_KEY"] = CONFIG.api_key

import phoenix as px
import pandas as pd

from phoenix.trace.dsl import SpanQuery
from phoenix.trace import SpanEvaluations, using_project

In [19]:
pd.set_option("display.max_colwidth", None)

def normalize_newline(s: str) -> str:
    """Chuyển '\\n' literal thành newline thật."""
    return s.replace("\\n", "\n") if isinstance(s, str) else s

def split_ref_items(s: str):
    """
    Tách chuỗi ref dạng:
    [1] Intel focuses...
    [2] AMD Ryzen offers...
    [4] Laptop battery life...
    Thành list [(id, text), ...]
    """
    if not isinstance(s, str):
        return []
    s = normalize_newline(s)
    pattern = re.compile(r"\[(\d+)\]\s*(.*?)(?=(?:\n\[\d+\])|$)", flags=re.S)
    return pattern.findall(s)

def explode_refs(df: pd.DataFrame, ref_col: str = "ref") -> pd.DataFrame:
    """
    Nhận DataFrame có cột 'ref', trả về DataFrame chỉ gồm context.trace_id và ref_text.
    """
    tmp = df.copy()
    tmp["ref_items"] = tmp[ref_col].apply(split_ref_items)
    out = tmp.explode("ref_items", ignore_index=True)
    out[["ref_id", "ref_text"]] = pd.DataFrame(out["ref_items"].tolist(), index=out.index)
    out["ref_id"] = out["ref_id"].astype(int)
    return out[["ref_text"]]

In [20]:
import phoenix as px
from phoenix.trace.dsl import SpanQuery

query = SpanQuery().where("span_kind == 'CHAIN'", ).select("trace_id", input="input.value", output="output.value")

df = px.Client().query_spans(query, project_name="hugging-face")

reference = SpanQuery().where("span_kind == 'TOOL'").select("trace_id", ref="prompt.context.preview")
spans_with_docs_df = px.Client().query_spans(reference, project_name="hugging-face")
print(len(spans_with_docs_df))

document_chunks_df = explode_refs(spans_with_docs_df)
print("Exploded format:\n", document_chunks_df)

2
Exploded format:
                                                                                                                             ref_text
0    the more affordable choice, but in recent years, some of its flagship products even surpassed Intel’s in both price and raw per
1   its efficiency cores and hybrid architecture, especially with the Alder Lake and Raptor Lake generations. "\n    "The launch of 
2   dvanced process nodes to fabricate its CPUs and GPUs, allowing it to deliver competitive products without owning its own fabs. "
3        "AMD's Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat
4  nerations. "\n    "The launch of AMD's EPYC server processors disrupted the data center market, offering higher core counts and b
5  nd pre-built PCs. "\n    "Both companies are expanding into GPU technologies, with AMD focusing on Radeon graphics and Intel laun


In [21]:
generate_questions_template = """\
Context information is below.

---------------------
{ref_text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

In [22]:
def normalize_newline(s: str) -> str:
    """Chuyển '\\n' literal thành newline thật."""
    return s.replace("\\n", "\n") if isinstance(s, str) else s

def split_ref_items(s: str):
    """
    Tách chuỗi ref dạng:
    [1] Intel focuses...
    [2] AMD Ryzen offers...
    [4] Laptop battery life...
    Thành list [(id, text), ...]
    """
    if not isinstance(s, str):
        return []
    s = normalize_newline(s)
    pattern = re.compile(r"\[(\d+)\]\s*(.*?)(?=(?:\n\[\d+\])|$)", flags=re.S)
    return pattern.findall(s)

def explode_refs(df: pd.DataFrame, ref_col: str = "ref") -> pd.DataFrame:
    """
    Nhận DataFrame có cột 'ref', trả về DataFrame chỉ gồm context.trace_id và ref_text.
    """
    tmp = df.copy()
    tmp["ref_items"] = tmp[ref_col].apply(split_ref_items)
    out = tmp.explode("ref_items", ignore_index=True)
    out[["ref_id", "ref_text"]] = pd.DataFrame(out["ref_items"].tolist(), index=out.index)
    out["ref_id"] = out["ref_id"].astype(int)
    return out[["context.trace_id", "ref_text"]]

In [23]:
def robust_output_parser(response: str, index: int):
    s = (response or "").strip()
    # log vài mẫu đầu để bạn xem raw
    if index < 3:
        print(f"\nRAW[{index}]:\n{repr(s)}\n")

    if not s:
        return {"__error__": "empty", "question_1": None, "question_2": None, "question_3": None}

    # 1) bỏ code fences ```json ... ```
    if s.startswith("```"):
        s = re.sub(r"^```[a-zA-Z]*\s*", "", s)
        s = re.sub(r"\s*```$", "", s, flags=re.S)

    # 2) lấy block JSON đầu tiên
    start, end = s.find("{"), s.rfind("}")
    if start != -1 and end != -1 and end > start:
        try:
            obj = json.loads(s[start:end+1])
            # đảm bảo trả về dict có các key mong muốn
            return {
                "question_1": obj.get("question_1"),
                "question_2": obj.get("question_2"),
                "question_3": obj.get("question_3"),
            }
        except json.JSONDecodeError as e:
            pass

    # 3) fallback: nếu model trả plain text, nhét vào question_1
    return {"question_1": s, "question_2": None, "question_3": None}

In [25]:
model = LiteLLMModel(
    model="huggingface/together/Qwen/Qwen2.5-7B-Instruct",
    temperature=0.0,
)

questions_df = llm_generate(
    dataframe=document_chunks_df,
    template=generate_questions_template,
    model=model,
    output_parser=robust_output_parser,
    concurrency=20,
)

🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
llm_generate |          | 0/6 (0.0%) | ⏳ 00:00<? | ?it/s20:26:59 - LiteLLM:DEBUG: utils.py:349 - 

20:26:59 - LiteLLM:DEBUG: utils.py:349 - Request to litellm:
20:26:59 - LiteLLM:DEBUG: utils.py:349 - litellm.completion(model='huggingface/together/Qwen/Qwen2.5-7B-Instruct', messages=[{'content': 'Context information is below.\n\n---------------------\nthe more affordable choice, but in recent years, some of its flagship products even surpassed Intel’s in both price and raw per\n---------------------\n\nGiven the context information and not prior knowledge.\ngenerate only questions based on the below query.\n\nYou are a Teacher/ Professor. Your task is to setup 3 questions for an upcoming quiz/examination. The questions should be diverse in nature across the document. Restrict the questions 


RAW[0]:
'```json\n{\n  "question_1": "According to the context, which company\'s flagship products have recently surpassed Intel\'s in terms of both price and performance?",\n  "question_2": "What does the phrase \'the more affordable choice\' likely refer to in the given context?",\n  "question_3": "Based on the information provided, what trend can be observed in the competitive landscape between the two companies mentioned?"\n}\n```'



`max_retries` is not supported. It will be ignored.
20:27:01 - LiteLLM:DEBUG: litellm_logging.py:929 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://router.huggingface.co/together/v1/chat/completions \
-H 'content-type: application/json' -H 'Authorization: Be****Hh' \
-d '{'model': 'Qwen/Qwen2.5-7B-Instruct-Turbo', 'messages': [{'content': 'Context information is below.\n\n---------------------\nits efficiency cores and hybrid architecture, especially with the Alder Lake and Raptor Lake generations. "\n    "The launch of \n---------------------\n\nGiven the context information and not prior knowledge.\ngenerate only questions based on the below query.\n\nYou are a Teacher/ Professor. Your task is to setup 3 questions for an upcoming quiz/examination. The questions should be diverse in nature across the document. Restrict the questions to the context information provided."\n\nOutput the questions in JSON format with the keys question_1, question_2, question_3.\n', 'role': 'us


RAW[1]:
'```json\n{\n  "question_1": "What are the two specific processor generations mentioned in the context that feature efficiency cores and a hybrid architecture?",\n  "question_2": "Identify the term used in the context to describe the combination of different types of processor cores in these generations.",\n  "question_3": "Based on the context, which event is indicated to have occurred with the mention of \'The launch of\'?"\n}\n```'



`max_retries` is not supported. It will be ignored.
20:27:03 - LiteLLM:DEBUG: litellm_logging.py:929 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://router.huggingface.co/together/v1/chat/completions \
-H 'content-type: application/json' -H 'Authorization: Be****Hh' \
-d '{'model': 'Qwen/Qwen2.5-7B-Instruct-Turbo', 'messages': [{'content': 'Context information is below.\n\n---------------------\ndvanced process nodes to fabricate its CPUs and GPUs, allowing it to deliver competitive products without owning its own fabs. "\n---------------------\n\nGiven the context information and not prior knowledge.\ngenerate only questions based on the below query.\n\nYou are a Teacher/ Professor. Your task is to setup 3 questions for an upcoming quiz/examination. The questions should be diverse in nature across the document. Restrict the questions to the context information provided."\n\nOutput the questions in JSON format with the keys question_1, question_2, question_3.\n', 'role': 'us


RAW[2]:
'```json\n{\n  "question_1": "What does the term \'advanced process nodes\' refer to in the context of CPU and GPU fabrication?",\n  "question_2": "Why might a company choose not to own its own fabrication plants (fabs) when producing CPUs and GPUs?",\n  "question_3": "How does using advanced process nodes contribute to delivering competitive products in the CPU and GPU market?"\n}\n```'



`max_retries` is not supported. It will be ignored.
20:27:04 - LiteLLM:DEBUG: litellm_logging.py:929 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://router.huggingface.co/together/v1/chat/completions \
-H 'content-type: application/json' -H 'Authorization: Be****Hh' \
-d '{'model': 'Qwen/Qwen2.5-7B-Instruct-Turbo', 'messages': [{'content': 'Context information is below.\n\n---------------------\n"AMD\'s Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat\n---------------------\n\nGiven the context information and not prior knowledge.\ngenerate only questions based on the below query.\n\nYou are a Teacher/ Professor. Your task is to setup 3 questions for an upcoming quiz/examination. The questions should be diverse in nature across the document. Restrict the questions to the context information provided."\n\nOutput the questions in JSON format with the keys question_1, question_2, question_3.\n', 'role': 'user'}

Process was interrupted. The return value will be incomplete...


In [13]:
questions_df.head()

,question_1,question_2,question_3
0,What feature of AMD's Ryzen processors has contributed to their popularity among content creators?,How does the multi-core performance of AMD's Ryzen processors benefit content creation tasks?,"Based on the context, which type of users are likely to find AMD's Ryzen processors particularly appealing?"
1,What was the impact of AMD's EPYC server processors on the data center market according to the given context?,"Based on the information provided, what specific feature of AMD's EPYC processors is mentioned as a key factor in disrupting the market?",Summarize the main point of the given context in one sentence.
2,What are the two companies mentioned in the context that are expanding into GPU technologies?,Which company is focusing on Radeon graphics according to the context?,What is the focus of Intel's expansion into GPU technologies based on the given information?
3,"According to the context, what are both AMD and Intel focusing on in the future?",How does the context describe the current state of price and raw performance between AMD and Intel?,Summarize the main points mentioned about AMD and Intel's future strategies in the given context.
4,"What does the term 'fabs' refer to in the context of semiconductor manufacturing, and why might a company like AMD not own its own fabs?","How has AMD's performance in the desktop market changed in recent years, and what does the context suggest about Intel's position in this market?","Based on the given context, what can be inferred about the competitive landscape between AMD and Intel in the desktop market?"


In [14]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, document_chunks_df], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["ref_text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

In [15]:
questions_with_document_chunk_df.head(10)

,ref_text,question
0,"""AMD's Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat",What feature of AMD's Ryzen processors has contributed to their popularity among content creators?
1,"nerations. ""\n ""The launch of AMD's EPYC server processors disrupted the data center market, offering higher core counts and b",What was the impact of AMD's EPYC server processors on the data center market according to the given context?
2,"nd pre-built PCs. ""\n ""Both companies are expanding into GPU technologies, with AMD focusing on Radeon graphics and Intel laun",What are the two companies mentioned in the context that are expanding into GPU technologies?
3,"ntel’s in both price and raw performance. ""\n ""Looking forward, both AMD and Intel are betting on AI, high-performance computi","According to the context, what are both AMD and Intel focusing on in the future?"
4,"s without owning its own fabs. ""\n ""While AMD gained significant momentum in the desktop market, Intel still maintains strong","What does the term 'fabs' refer to in the context of semiconductor manufacturing, and why might a company like AMD not own its own fabs?"
5,"nd pre-built PCs. ""\n ""Both companies are expanding into GPU technologies, with AMD focusing on Radeon graphics and Intel laun",What are the two companies mentioned in the context that are expanding into GPU technologies?
6,"""AMD's Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat",What feature of AMD's Ryzen processors has contributed to their popularity among content creators?
7,"nerations. ""\n ""The launch of AMD's EPYC server processors disrupted the data center market, offering higher core counts and b",What was the impact of AMD's EPYC server processors on the data center market according to the given context?
8,"el has faced challenges with delays in adopting smaller nanometer processes, while AMD leveraged TSMC’s 7nm and 5nm technology t",What challenge has el faced in adopting newer nanometer processes?
9,"""AMD's Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat",How does the multi-core performance of AMD's Ryzen processors benefit content creation tasks?


In [ ]:
# import os
# import pandas as pd
# from tqdm import tqdm
# from opentelemetry.trace import Status, StatusCode
# from Phoenix.trace.tracing import tracer

# from LiteLLM.lite import LiteLLMClient
# from LiteLLM.Response import ResponseInput
# from tools.rag import build_prompt

# # --------- CONFIG ---------
# # df nguồn: lấy cột "question"
# SOURCE_DF = questions_with_document_chunk_df   # <- đã có sẵn ở phía bạn
# QUESTION_COL = "question"
# TOP_K = 3          # build_prompt(query, top_k=TOP_K)
# MODEL = os.getenv("LLM_MODEL", "gpt-4o-mini")  # đổi nếu cần
# TEMP = float(os.getenv("LLM_TEMP", "0.2"))
# MAX_NUM = None     # giới hạn số câu chạy (None = chạy hết)
# # --------------------------

# def main():
#     # Chuẩn bị danh sách câu hỏi
#     queries = (
#         SOURCE_DF[QUESTION_COL]
#         .dropna()
#         .map(lambda s: str(s).strip())
#         .loc[lambda s: s.ne("")]
#         .tolist()
#     )
#     if MAX_NUM is not None:
#         queries = queries[:MAX_NUM]

#     print(f"Running {len(queries)} questions…")

#     # Client LLM (tuỳ môi trường bạn cấu hình API key/endpoint)
#     client = LiteLLMClient()

#     rows = []  # lưu kết quả

#     for i, query in enumerate(tqdm(queries, desc="RAG batch", unit="q")):
#         with tracer.start_as_current_span("Thought") as span:
#             span.set_attribute("openinference.span.kind", "CHAIN")
#             span.set_attribute("input.value", query)

#             try:
#                 # tạo prompt từ RAG (lấy context top-k cho câu hỏi)
#                 prompt = build_prompt(query, top_k=TOP_K)

#                 # gọi model
#                 msg = ResponseInput(prompt)
#                 resp = client.complete([msg])           # one-by-one (ổn định)
#                 answer = resp.transform()
#                 usage = None
#                 try:
#                     usage = resp.usage()
#                 except Exception:
#                     pass

#                 # log lên trace
#                 span.set_attribute("output.value", (answer or "")[:400])
#                 span.set_status(Status(StatusCode.OK))

#                 rows.append({
#                     "index": i,
#                     "question": query,
#                     "prompt": prompt,
#                     "answer": answer,
#                     "usage": usage,
#                     "error": None,
#                 })

#             except Exception as e:
#                 # ghi lỗi nhưng không dừng batch
#                 span.record_exception(e)
#                 span.set_status(Status(StatusCode.ERROR, str(e)))
#                 rows.append({
#                     "index": i,
#                     "question": query,
#                     "prompt": None,
#                     "answer": None,
#                     "usage": None,
#                     "error": str(e),
#                 })

#     # Thành DataFrame kết quả
#     results_df = pd.DataFrame(rows)
#     pd.set_option("display.max_colwidth", None)
#     print("\n=== SAMPLE RESULTS ===")
#     print(results_df.head(10)[["question", "answer", "error"]])

#     # (tuỳ chọn) lưu file
#     # results_df.to_csv("rag_batch_results.csv", index=False)
#     return results_df

# if __name__ == "__main__":
#     _ = main()


OpenTelemetry Tracing Details
|  Phoenix Project: hugging-face
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

Running 27 questions…


RAG batch:   0%|          | 0/27 [00:12<?, ?q/s]


KeyboardInterrupt: 

In [26]:
client = px.Client()

query = (
    SpanQuery()
    .where("span_kind == 'RETRIEVER' and evals['relevance'].label is None   ")
    .select("context.span_id", "context.trace_id", "input.value", "retrieval.documents")
)

df = client.query_spans(query, project_name="hugging-face")
df = df.reset_index().rename(columns={"index": "context.span_id"})
print(df.columns)
print(df.head())

Index(['context.span_id', 'context.trace_id', 'input.value',
       'retrieval.documents'],
      dtype='object')
    context.span_id                  context.trace_id  \
0  99ada09c725d14af  79a556783429c63d0be2f78cc668e5b2   
1  0a46c739623afaf8  8c37e9fe85f8bf62b84186e36a95eece   
2  c149e18e115a10db  012ba572ab94742d40164868224f2599   
3  18aee44228d90a80  6027ca1f7c45cb40e05605d3b6bb73b5   

                                 input.value  \
0  Can you give me any details about Samsung   
1       I want to know information about AMD   
2            I want to compare AMD and Intel   
3                      Is AMD has multi core   

                                                                                                                                                                                                                                                                                                                                                                       

In [27]:
df_exploded = df.explode("retrieval.documents", ignore_index=True)

print(df_exploded.columns)

Index(['context.span_id', 'context.trace_id', 'input.value',
       'retrieval.documents'],
      dtype='object')


In [28]:
# normalize nested dict
docs = pd.json_normalize(df_exploded["retrieval.documents"])

# gộp lại với trace_id
retrieved_documents_df = pd.concat([df_exploded.drop(columns=["retrieval.documents"]), docs], axis=1)

print(retrieved_documents_df.columns)

Index(['context.span_id', 'context.trace_id', 'input.value', 'document.id',
       'document.score', 'document.content'],
      dtype='object')


In [29]:
retrieved_documents_df = retrieved_documents_df.rename(
    columns={
        "document.content": "reference",
        "input.value": "input"   # nếu bạn có cột input.value thì rename
    }
)

In [21]:
retrieved_documents_df

,context.span_id,context.trace_id,input,reference,document.id,document.score
0,6779f6adbac89657,988c1a51b760595beade111fa37c03bb,I want to know information about AMD,"\n ""AMD's Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat",6e13a8d5-0deb-4982-8f6a-601164f66a73,0.025658
1,6779f6adbac89657,988c1a51b760595beade111fa37c03bb,I want to know information about AMD,"nerations. ""\n ""The launch of AMD's EPYC server processors disrupted the data center market, offering higher core counts and b",825177e1-6c77-4de7-bb02-c332e8dbaf54,0.007090
2,6779f6adbac89657,988c1a51b760595beade111fa37c03bb,I want to know information about AMD,"nd pre-built PCs. ""\n ""Both companies are expanding into GPU technologies, with AMD focusing on Radeon graphics and Intel laun",a23b9fae-a936-451e-b8d5-573ab30f08dd,0.004895
3,a0ec6243d4457141,a2e42b61b13ad2a0b66de97ce42993e5,I want some compare between AMD and Intel,"ntel’s in both price and raw performance. ""\n ""Looking forward, both AMD and Intel are betting on AI, high-performance computi",def89f0c-d7d3-4c57-964a-4689cfda6ad3,0.830985
4,a0ec6243d4457141,a2e42b61b13ad2a0b66de97ce42993e5,I want some compare between AMD and Intel,"s without owning its own fabs. ""\n ""While AMD gained significant momentum in the desktop market, Intel still maintains strong",207f6d67-3dbf-4aea-88dc-c9df94f43b43,0.743740
5,a0ec6243d4457141,a2e42b61b13ad2a0b66de97ce42993e5,I want some compare between AMD and Intel,"nd pre-built PCs. ""\n ""Both companies are expanding into GPU technologies, with AMD focusing on Radeon graphics and Intel laun",a23b9fae-a936-451e-b8d5-573ab30f08dd,0.736834
6,0e559a01e10f0f40,4c8aaf9e347a8e0c08b9a196e00b4347,Is AMD has multi core,"\n ""AMD's Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat",6e13a8d5-0deb-4982-8f6a-601164f66a73,0.993957
7,0e559a01e10f0f40,4c8aaf9e347a8e0c08b9a196e00b4347,Is AMD has multi core,"nerations. ""\n ""The launch of AMD's EPYC server processors disrupted the data center market, offering higher core counts and b",825177e1-6c77-4de7-bb02-c332e8dbaf54,0.380465
8,0e559a01e10f0f40,4c8aaf9e347a8e0c08b9a196e00b4347,Is AMD has multi core,"el has faced challenges with delays in adopting smaller nanometer processes, while AMD leveraged TSMC’s 7nm and 5nm technology t",db538914-83d3-45b7-81e9-6cdd203b2c7c,0.027638
9,0c8c078f8afc59e3,572ce6d156e7bec8e0bf40c24b06954b,What feature of AMD's Ryzen processors has contributed to their popularity among content creators?,NaN,NaN,NaN


In [30]:
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
)

relevance_evaluator = RelevanceEvaluator(model)

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.
run_evals |          | 0/12 (0.0%) | ⏳ 00:00<? | ?it/s20:28:03 - LiteLLM:DEBUG: utils.py:349 - 

20:28:03 - LiteLLM:DEBUG: utils.py:349 - Request to litellm:
20:28:03 - LiteLLM:DEBUG: utils.py:349 - litellm.completion(model='huggingface/together/Qwen/Qwen2.5-7B-Instruct', messages=[{'content': '\nYou are comparing a reference text to a question and trying to determine if the reference text\ncontains information relevant to answering the question. Here is the data:\n    [BEGIN DATA]\n    ************\n    [Question]: Can you give me any details about Samsung\n    ************\n    [Reference text]:  the more affordable choice, but in recent years, some of its flagship products even surpassed Intel’s in both price and raw per\n    ************\n    [END DATA]\nCompare the Question above to th


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 1: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
Retrying...


20:28:19 - LiteLLM:DEBUG: litellm_logging.py:474 - self.optional_params: {}
20:28:19 - LiteLLM:DEBUG: utils.py:349 - SYNC kwargs[caching]: False; litellm.cache: None; kwargs.get('cache')['no-cache']: False
20:28:19 - LiteLLM:INFO: utils.py:3309 - 
LiteLLM completion() model= together/Qwen/Qwen2.5-7B-Instruct; provider = huggingface
20:28:19 - LiteLLM:DEBUG: utils.py:3312 - 
LiteLLM: Params passed to completion() {'model': 'together/Qwen/Qwen2.5-7B-Instruct', 'functions': None, 'function_call': None, 'temperature': 0.0, 'top_p': 1, 'n': None, 'stream': None, 'stream_options': None, 'stop': None, 'max_tokens': 1024, 'max_completion_tokens': None, 'modalities': None, 'prediction': None, 'audio': None, 'presence_penalty': None, 'frequency_penalty': None, 'logit_bias': None, 'user': None, 'custom_llm_provider': 'huggingface', 'response_format': None, 'seed': None, 'tools': None, 'tool_choice': None, 'max_retries': 0, 'logprobs': None, 'top_logprobs': None, 'extra_headers': None, 'api_versio


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 2: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
Retrying...


`max_retries` is not supported. It will be ignored.
20:28:20 - LiteLLM:DEBUG: litellm_logging.py:929 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://router.huggingface.co/together/v1/chat/completions \
-H 'content-type: application/json' -H 'Authorization: Be****Hh' \
-d '{'model': 'Qwen/Qwen2.5-7B-Instruct-Turbo', 'messages': [{'content': '\nYou are comparing a reference text to a question and trying to determine if the reference text\ncontains information relevant to answering the question. Here is the data:\n    [BEGIN DATA]\n    ************\n    [Question]: Is AMD has multi core\n    ************\n    [Reference text]: nerations. "\n    "The launch of AMD\'s EPYC server processors disrupted the data center market, offering higher core counts and b\n    ************\n    [END DATA]\nCompare the Question above to the Reference text. You must determine whether the Reference text\ncontains information that can help answer the Question. First, write out in a step by step man

...


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 9: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
Retrying...


`max_retries` is not supported. It will be ignored.
20:28:25 - LiteLLM:DEBUG: litellm_logging.py:929 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://router.huggingface.co/together/v1/chat/completions \
-H 'content-type: application/json' -H 'Authorization: Be****Hh' \
-d '{'model': 'Qwen/Qwen2.5-7B-Instruct-Turbo', 'messages': [{'content': '\nYou are comparing a reference text to a question and trying to determine if the reference text\ncontains information relevant to answering the question. Here is the data:\n    [BEGIN DATA]\n    ************\n    [Question]: Is AMD has multi core\n    ************\n    [Reference text]: nerations. "\n    "The launch of AMD\'s EPYC server processors disrupted the data center market, offering higher core counts and b\n    ************\n    [END DATA]\nCompare the Question above to the Reference text. You must determine whether the Reference text\ncontains information that can help answer the Question. First, write out in a step by step man


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

Exception in worker on attempt 10: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}
Retrying...


`max_retries` is not supported. It will be ignored.
20:28:26 - LiteLLM:DEBUG: litellm_logging.py:929 - 

POST Request Sent from LiteLLM:
curl -X POST \
https://router.huggingface.co/together/v1/chat/completions \
-H 'content-type: application/json' -H 'Authorization: Be****Hh' \
-d '{'model': 'Qwen/Qwen2.5-7B-Instruct-Turbo', 'messages': [{'content': '\nYou are comparing a reference text to a question and trying to determine if the reference text\ncontains information relevant to answering the question. Here is the data:\n    [BEGIN DATA]\n    ************\n    [Question]: Is AMD has multi core\n    ************\n    [Reference text]: nerations. "\n    "The launch of AMD\'s EPYC server processors disrupted the data center market, offering higher core counts and b\n    ************\n    [END DATA]\nCompare the Question above to the Reference text. You must determine whether the Reference text\ncontains information that can help answer the Question. First, write out in a step by step man


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

Retries exhausted after 11 attempts: litellm.APIError: HuggingfaceException - {"error":"You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits."}


In [23]:
retrieved_documents_relevance_df.head()

,label,score,explanation
0,relevant,1,EXPLANATION: The question asks for information about AMD. The reference text mentions AMD's Ryzen processors and provides specific information about their performance. This information can help answer the question by providing details about AMD's products and their capabilities.\n\nLABEL: relevant
1,relevant,1,"EXPLANATION: The question asks for information about AMD. The reference text mentions AMD's EPYC server processors and their impact on the data center market. This information is directly related to AMD's products and market presence, which can help answer the question about AMD.\nLABEL: relevant"
2,relevant,1,"EXPLANATION: The question asks for information about AMD. The reference text mentions AMD specifically, noting its focus on Radeon graphics. This information is directly relevant to answering the question about AMD.\nLABEL: relevant"
3,relevant,1,"EXPLANATION: The question asks for a comparison between AMD and Intel. The reference text mentions both companies and provides a brief comparison in terms of price and raw performance. It also mentions their future focus areas. While the text does not provide a detailed comparison, it does contain relevant information that can help answer the question.\n\nLABEL: relevant"
4,relevant,1,"EXPLANATION: The question asks for a comparison between AMD and Intel. The reference text mentions both AMD and Intel, discussing AMD's momentum in the desktop market and Intel's strong position. This information can be used to compare the two companies, even though the text does not explicitly state a comparison. Therefore, the reference text contains relevant information to answer the question.\n\nLABEL: ""relevant"""


In [31]:
retrieved_documents_df.columns

Index(['context.span_id', 'context.trace_id', 'input', 'document.id',
       'document.score', 'reference'],
      dtype='object')

In [32]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)
documents_with_relevance_df

,context.span_id,context.trace_id,input,document.id,document.score,reference,eval_label,eval_score,eval_explanation
0,99ada09c725d14af,79a556783429c63d0be2f78cc668e5b2,Can you give me any details about Samsung,09b3a531-26e4-4023-8b0a-ae6af3affc40,0.001939,"the more affordable choice, but in recent years, some of its flagship products even surpassed Intel’s in both price and raw per",relevant,1.0,"EXPLANATION: The question asks for details about Samsung. The reference text mentions ""flagship products"" related to Samsung, even though it does not provide specific details. However, it does indicate that Samsung's products have competed with Intel's in terms of price and performance. This information, while not detailed, does provide some context about Samsung's product range and market position.\nLABEL: relevant"
1,99ada09c725d14af,79a556783429c63d0be2f78cc668e5b2,Can you give me any details about Samsung,a0149ea1-fb03-4f53-be3a-891702d05314,0.000306,"its efficiency cores and hybrid architecture, especially with the Alder Lake and Raptor Lake generations. ""\n ""The launch of",unrelated,0.0,"EXPLANATION: The question asks for details about Samsung. The reference text does not mention Samsung at all, instead it discusses efficiency cores and hybrid architecture in the context of processor generations like Alder Lake and Raptor Lake. This information is not relevant to Samsung as a company or brand.\nLABEL: unrelated"
2,99ada09c725d14af,79a556783429c63d0be2f78cc668e5b2,Can you give me any details about Samsung,bb1046e5-feb9-4bb8-8546-95829d996604,0.000189,"dvanced process nodes to fabricate its CPUs and GPUs, allowing it to deliver competitive products without owning its own fabs. """,relevant,1.0,"EXPLANATION: The question asks for details about Samsung. The reference text mentions Samsung's process of fabricating CPUs and GPUs using advanced process nodes and not owning its own fabs. This information provides insight into Samsung's manufacturing strategy, which can be considered relevant details about the company. However, the text does not provide a comprehensive overview of Samsung, so it is limited in scope.\n\nLABEL: ""relevant"""
3,0a46c739623afaf8,8c37e9fe85f8bf62b84186e36a95eece,I want to know information about AMD,6e13a8d5-0deb-4982-8f6a-601164f66a73,0.025658,"\n ""AMD's Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat",relevant,1.0,EXPLANATION: The question asks for information about AMD. The reference text mentions AMD's Ryzen processors and provides specific information about their performance. This information can help answer the question by providing details about AMD's products and their capabilities.\nLABEL: relevant
4,0a46c739623afaf8,8c37e9fe85f8bf62b84186e36a95eece,I want to know information about AMD,825177e1-6c77-4de7-bb02-c332e8dbaf54,0.007090,"nerations. ""\n ""The launch of AMD's EPYC server processors disrupted the data center market, offering higher core counts and b",relevant,1.0,"EXPLANATION: The question asks for information about AMD. The reference text mentions AMD's EPYC server processors and their impact on the data center market. This information is directly related to AMD's products and market presence, which can help answer the question about AMD.\nLABEL: relevant"
5,0a46c739623afaf8,8c37e9fe85f8bf62b84186e36a95eece,I want to know information about AMD,a23b9fae-a936-451e-b8d5-573ab30f08dd,0.004895,"nd pre-built PCs. ""\n ""Both companies are expanding into GPU technologies, with AMD focusing on Radeon graphics and Intel laun",relevant,1.0,"EXPLANATION: The question asks for information about AMD. The reference text mentions AMD specifically, noting its focus on Radeon graphics. This information is directly relevant to answering the question about AMD.\nLABEL: relevant"
6,c149e18e115a10db,012ba572ab94742d40164868224f2599,I want to compare AMD and Intel,def89f0c-d7d3-4c57-964a-4689cfda6ad3,0.857612,"ntel’s in both price and raw p

In [41]:
documents_with_relevance_df = documents_with_relevance_df.rename(
    columns={
        "eval_label": "label",
        "eval_score": "score",
        "eval_explanation": "explanation",
        "document.score": "document_score",
    }
)

print(documents_with_relevance_df.head())

    context.span_id                  context.trace_id  \
0  99ada09c725d14af  79a556783429c63d0be2f78cc668e5b2   
1  99ada09c725d14af  79a556783429c63d0be2f78cc668e5b2   
2  99ada09c725d14af  79a556783429c63d0be2f78cc668e5b2   
3  0a46c739623afaf8  8c37e9fe85f8bf62b84186e36a95eece   
4  0a46c739623afaf8  8c37e9fe85f8bf62b84186e36a95eece   

                                       input  \
0  Can you give me any details about Samsung   
1  Can you give me any details about Samsung   
2  Can you give me any details about Samsung   
3       I want to know information about AMD   
4       I want to know information about AMD   

                            document.id  document_score  \
0  09b3a531-26e4-4023-8b0a-ae6af3affc40        0.001939   
1  a0149ea1-fb03-4f53-be3a-891702d05314        0.000306   
2  bb1046e5-feb9-4bb8-8546-95829d996604        0.000189   
3  6e13a8d5-0deb-4982-8f6a-601164f66a73        0.025658   
4  825177e1-6c77-4de7-bb02-c332e8dbaf54        0.007090   

             

In [42]:
documents_with_relevance_df

,context.span_id,context.trace_id,input,document.id,document_score,reference,label,score,explanation
0,99ada09c725d14af,79a556783429c63d0be2f78cc668e5b2,Can you give me any details about Samsung,09b3a531-26e4-4023-8b0a-ae6af3affc40,0.001939,"the more affordable choice, but in recent years, some of its flagship products even surpassed Intel’s in both price and raw per",relevant,1.0,"EXPLANATION: The question asks for details about Samsung. The reference text mentions ""flagship products"" related to Samsung, even though it does not provide specific details. However, it does indicate that Samsung's products have competed with Intel's in terms of price and performance. This information, while not detailed, does provide some context about Samsung's product range and market position.\nLABEL: relevant"
1,99ada09c725d14af,79a556783429c63d0be2f78cc668e5b2,Can you give me any details about Samsung,a0149ea1-fb03-4f53-be3a-891702d05314,0.000306,"its efficiency cores and hybrid architecture, especially with the Alder Lake and Raptor Lake generations. ""\n ""The launch of",unrelated,0.0,"EXPLANATION: The question asks for details about Samsung. The reference text does not mention Samsung at all, instead it discusses efficiency cores and hybrid architecture in the context of processor generations like Alder Lake and Raptor Lake. This information is not relevant to Samsung as a company or brand.\nLABEL: unrelated"
2,99ada09c725d14af,79a556783429c63d0be2f78cc668e5b2,Can you give me any details about Samsung,bb1046e5-feb9-4bb8-8546-95829d996604,0.000189,"dvanced process nodes to fabricate its CPUs and GPUs, allowing it to deliver competitive products without owning its own fabs. """,relevant,1.0,"EXPLANATION: The question asks for details about Samsung. The reference text mentions Samsung's process of fabricating CPUs and GPUs using advanced process nodes and not owning its own fabs. This information provides insight into Samsung's manufacturing strategy, which can be considered relevant details about the company. However, the text does not provide a comprehensive overview of Samsung, so it is limited in scope.\n\nLABEL: ""relevant"""
3,0a46c739623afaf8,8c37e9fe85f8bf62b84186e36a95eece,I want to know information about AMD,6e13a8d5-0deb-4982-8f6a-601164f66a73,0.025658,"\n ""AMD's Ryzen processors have been praised for their strong multi-core performance, making them attractive for content creat",relevant,1.0,EXPLANATION: The question asks for information about AMD. The reference text mentions AMD's Ryzen processors and provides specific information about their performance. This information can help answer the question by providing details about AMD's products and their capabilities.\nLABEL: relevant
4,0a46c739623afaf8,8c37e9fe85f8bf62b84186e36a95eece,I want to know information about AMD,825177e1-6c77-4de7-bb02-c332e8dbaf54,0.007090,"nerations. ""\n ""The launch of AMD's EPYC server processors disrupted the data center market, offering higher core counts and b",relevant,1.0,"EXPLANATION: The question asks for information about AMD. The reference text mentions AMD's EPYC server processors and their impact on the data center market. This information is directly related to AMD's products and market presence, which can help answer the question about AMD.\nLABEL: relevant"
5,0a46c739623afaf8,8c37e9fe85f8bf62b84186e36a95eece,I want to know information about AMD,a23b9fae-a936-451e-b8d5-573ab30f08dd,0.004895,"nd pre-built PCs. ""\n ""Both companies are expanding into GPU technologies, with AMD focusing on Radeon graphics and Intel laun",relevant,1.0,"EXPLANATION: The question asks for information about AMD. The reference text mentions AMD specifically, noting its focus on Radeon graphics. This information is directly relevant to answering the question about AMD.\nLABEL: relevant"
6,c149e18e115a10db,012ba572ab94742d40164868224f2599,I want to compare AMD and Intel,def89f0c-d7d3-4c57-964a-4689cfda6ad3,0.857612,"ntel’s in both price and raw performance. ""\

In [44]:
import numpy as np

final_df = documents_with_relevance_df.copy()

# ép score về số, biến lỗi thành NaN
final_df['__score__'] = pd.to_numeric(final_df['document_score'], errors='coerce')

# sort theo span_id và score giảm dần; NaN đẩy xuống cuối nhóm
final_df = final_df.sort_values(
    ['context.span_id', '__score__'], ascending=[True, False]
)

# đánh số 0,1,2,... trong từng span
final_df['document_position'] = final_df.groupby('context.span_id').cumcount()

# MultiIndex đúng chuẩn Phoenix (nếu bạn cần)
final_df = final_df.set_index(['context.span_id', 'document_position']).drop(columns='__score__')

In [45]:
final_df.columns

Index(['context.trace_id', 'input', 'document.id', 'document_score',
       'reference', 'label', 'score', 'explanation'],
      dtype='object')

In [47]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.score
    doc_scores[: len(df)] = df.document_score
    try:
        return ndcg_score([eval_scores], [doc_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": final_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

In [48]:
ndcg_at_2

,score
context.span_id,
0a46c739623afaf8,1.000000
18aee44228d90a80,NaN
99ada09c725d14af,0.613147
c149e18e115a10db,1.000000


In [50]:
precision_at_2 = pd.DataFrame(
    {
        "score": final_df.groupby("context.span_id").apply(
            lambda x: x.score[:2].sum(skipna=False) / 2
        )
    }
)

In [51]:
precision_at_2

,score
context.span_id,
0a46c739623afaf8,1.0
18aee44228d90a80,NaN
99ada09c725d14af,0.5
c149e18e115a10db,1.0


In [52]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.score[:2].sum(skipna=False) > 0
        )
    }
)

In [67]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=ndcg_at_2, eval_name="ndcg@2"),
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=final_df, eval_name="relevance"),
)

In [77]:
import phoenix as px

query = (
        SpanQuery()
        .where("span_kind == 'RETRIEVER' and evals['ndcg@2'].score is None")
        .select("context.span.id", "context.trace.id", "input.value", "retrieval.documents")
    )

df = px.Client().query_spans(query, project_name="hugging-face")
print(f"Found {len(df)} spans to evaluate")
print(df)

Found 1 spans to evaluate
                 context.span.id context.trace.id            input.value  \
context.span_id                                                            
18aee44228d90a80            None             None  Is AMD has multi core   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          retrieval.documents  
context.span_id          

In [17]:
import pandas as pd
import phoenix as px
from phoenix.trace.dsl import SpanQuery

PROJECT = "hugging-face"
client = px.Client()

# 1) Explode retrieved documents -> mỗi dòng = 1 doc
q_docs = (
    SpanQuery()
    .where("span_kind == 'RETRIEVER'")
    .select(input="input.value")
    .explode("retrieval.documents", reference="document.content")
)
docs = client.query_spans(q_docs, project_name=PROJECT)

# đem 2 cấp index ra cột thường để join
docs = docs.reset_index()  # có cột context.span_id, document_position
docs = docs.rename(columns={"context.span_id": "span_id"})

# 2) Lấy tất cả DocumentEvaluations tên 'relevance' đã log (span_id + document_position)
ev_list = client.get_evaluations(project_name=PROJECT)  # trả về list Evaluations
ev_parts = []
for ev in ev_list:
    df = getattr(ev, "dataframe", None)
    if df is None or ev.eval_name != "relevance":
        continue
    df = df.reset_index()  # lôi span_id, document_position ra cột
    if {"span_id", "document_position"}.issubset(df.columns):
        ev_parts.append(df[["span_id", "document_position"]])

evaluated = (
    pd.concat(ev_parts, ignore_index=True)
    if ev_parts else pd.DataFrame(columns=["span_id", "document_position"])
)

# 3) Anti-join -> chỉ giữ documents CHƯA được evaluate
unevaluated = (
    docs.merge(evaluated, on=["span_id", "document_position"], how="left", indicator=True)
        .query("_merge == 'left_only'")
        .drop(columns=["_merge"])
)

print(f"Documents CHƯA evaluate: {len(unevaluated)}")
unevaluated


Documents CHƯA evaluate: 6


,span_id,document_position,input,reference
0,99ada09c725d14af,0,Can you give me any details about Samsung,"the more affordable choice, but in recent yea..."
1,99ada09c725d14af,1,Can you give me any details about Samsung,"its efficiency cores and hybrid architecture,..."
2,99ada09c725d14af,2,Can you give me any details about Samsung,dvanced process nodes to fabricate its CPUs an...
3,0a46c739623afaf8,0,I want to know information about AMD,"\n ""AMD's Ryzen processors have been praise..."
4,0a46c739623afaf8,1,I want to know information about AMD,"nerations. ""\n ""The launch of AMD's EPYC se..."
5,0a46c739623afaf8,2,I want to know information about AMD,"nd pre-built PCs. ""\n ""Both companies are e..."
